## Notebook for spike sorting from .kwd data using the Pipeline of mountainsort (linux channel has pipeline)
Uses:
    - intan2kwik (https://github.com/zekearneodo/intan2kwik/blob/master/README.md)
    - mountainlab suite(https://github.com/flatironinstitute/mountainlab-js)
    - mountainsort https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md
    - mountainsort examples https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md

In [1]:
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
from importlib import reload

# mountainlab imports
from pipefinch.neural.convert import intan
from pipefinch.neural.sort.mountain import core as msc
from intan2kwik import kwd

import logging

# Setup the logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(formatter)
logger.addHandler(ch)
        
logger.info('Logger set')

/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
2018-12-19 15:54:35,245 - root - INFO - Logger set
2018-12-19 15:54:35,252 - matplotlib.backends - DEBUG - backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
os.environ['OMP_NUM_THREADS'] = '8'
os.environ['MKL_NUM_THREADS'] = '8'
os.environ['NUMEXPR_NUM_THREADS'] = '8'

### Session parameters and raw files

In [3]:
exp_base = os.path.abspath('/home/zinch/data/')
exp_mnt = os.path.abspath('/mnt/zuperfinchjr/Data/')

file_sys = {'exp_base': exp_base}

sess_par = {'bird': 'o9j9',
           'sess': '20181125_r021_saveontrigger',
            'neural': 'Neural'}


def file_names(sess_par, file_sys=None):
    f_s = unroll_file_sys(file_sys_defaults, file_sys)    
    
    f_n = {}
    f_n['raw'] = os.path.join(f_s['raw'], bird)

In [4]:
raw_folder = os.path.join(exp_mnt, sess_par['bird'], sess_par['neural'], sess_par['sess'])
ss_folder = os.path.join(exp_base, sess_par['bird'], 'ss_data', sess_par['sess'])

logger.info('Raw folder {}'.format(raw_folder))
#glob.glob(os.path.join(raw_folder, '*.rhd'))

2018-12-19 15:54:35,782 - root - INFO - Raw folder /mnt/zuperfinchjr/Data/o9j9/Neural/20181125_r021_saveontrigger


In [5]:
## Convert the whole session to a kwd file
kwd_path = os.path.join(ss_folder, 'experiment.kwd')
os.makedirs(ss_folder, exist_ok=True)
#first_intan_hdr, sess_pd = kwd.intan_to_kwd(raw_folder, kwd_path)

In [6]:
from pipefinch.h5tools.kwik import kwdfunctions as kwdf
from pipefinch.h5tools.kwik import kutil

## revisit reading a dataset

In [7]:
kwd_file = h5py.File(kwd_path, 'r')
dataset = kwd_file['/recordings/0/data']
print(dataset)

NameError: name 'h5py' is not defined

In [8]:
from pipefinch.h5tools.core import tables

In [9]:
all_dset = dataset.read_direct

NameError: name 'dataset' is not defined

In [10]:
# passing stuff to binary
reload(tables)

with open(bin_path, 'wb') as bf:
    stored = tables.dset_to_binary_file(dataset, bf, chan_list=[0])

NameError: name 'bin_path' is not defined

In [344]:
from mountainlab_pytools import mdaio
mdaio.DiskReadMda

<module 'mountainlab_pytools.mdaio' from '/home/zinch/conda/envs/msort/lib/python3.6/site-packages/mountainlab_pytools/mdaio/__init__.py'>

In [56]:
407574720/2

203787360.0

In [12]:
## Make .mda file with a set of recordings in a session
reload(kwdf)
reload(kutil)
ch.setLevel(logging.DEBUG)

all_rec_list = kutil.get_rec_list(kwd_path)
all_rec_list

chan_list = np.arange(5)

bin_path = os.path.join(ss_folder, 'experiment.bin')
bin_file = kwdf.kwd_to_binary(kwd_path, bin_path, chan_list=list(chan_list), rec_list=[0, 5, 10], header='mda')
rec_attr_dict = kwdf.get_rec_attrs(kwd_path, 0)
all_rec_meta = kwdf.get_all_rec_meta(kwd_path)

2018-12-19 15:55:20,588 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Writing kwd_file /home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/experiment.kwd to binary
2018-12-19 15:55:20,589 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Channels to extract: [0, 1, 2, 3, 4]
2018-12-19 15:55:20,776 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Will go through recs [0, 5, 10]
2018-12-19 15:55:20,777 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Creating binary file /home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/experiment.bin
2018-12-19 15:55:20,778 - pipefinch.h5tools.kwik.kwdfunctions - DEBUG - mda header


AttributeError: 'Dataset' object has no attribute 'dtpye'

In [354]:
all_rec_meta.loc[all_rec_meta['name'].isin([5, 15, 20]), 'samples_count'].sum()

1786320

In [352]:
ex_arr = kwdf.get_data_set(kwd_file, 0)
ex_arr.shape

(14556240, 14)

In [285]:
apd = pd.concat([pd.DataFrame([r[k] for r in all_rec_meta]) for k in all_rec_meta[0].keys()], axis=1)
apd.head()

,bit_depth,name,sample_rate,start_sample,start_time,channel_bit_volts,channel_names,channels_sample_rate,is_multiSampleRate_data,valid_samples
0,16,0,20000.0,0,1,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[14556240.0, 14556240.0, 14556240.0, 14556240...."
1,16,1,20000.0,0,1,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[106560.0, 106560.0, 106560.0, 106560.0, 10656..."
2,16,2,20000.0,0,1,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[1199520.0, 1199520.0, 1199520.0, 1199520.0, 1..."
3,16,3,20000.0,0,1,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[308160.0, 308160.0, 308160.0, 308160.0, 30816..."
4,16,4,20000.0,0,1,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[1199520.0, 1199520.0, 1199520.0, 1199520.0, 1..."


In [338]:
reload(kwdf)
cont, cont_pd = kwdf.check_continuity(apd)

In [339]:
cont

True

In [342]:
def check_continuity(meta_pd: dict, chan_idx_list=np.empty(0), rec_list=np.empty(0), check_fileds: list=['channel_names', 'channels_sample_rate']):

    rec_list = meta_pd['name'].values if rec_list.size ==0 else rec_list
    chan_idx_list = np.arange(meta_pd.loc[0, 'channel_names'].size) if chan_idx_list.size == 0 else chan_idx_list  
    rec_slice = meta_pd['name'].isin(rec_list)

    continuity_dict = {}
    for col_name in check_fileds:
        comp_list = kwdf.diff_array_columns(meta_pd.loc[rec_slice, :], col_name)

        comp_array = np.stack(comp_list[1:]) #first column of comp_list is a scalar True
        print(chan_idx_list)
        # comp_array is shape [rec_slice.size - 1, n_chan (tota)]
        cont_recs = np.all(comp_array[:, chan_idx_list], axis=1)
        continuity_dict[col_name] = cont_recs
    
    cont_pd = pd.DataFrame(continuity_dict)
    try:
        assert pd_check.values.all(), 'Error in continuity of recs'
    except:
        logger.warn('Errors in continuity of recordings')
        print(pd_ceck[pd_check.where(False)])
        logger.warn('')
        raise
    
pd_check = check_continuity(apd)

/home/zinch/conda/envs/msort/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
2018-12-19 13:06:34,896 - root - WARNING - Errors in continuity of recordings


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]


NameError: name 'pd_ceck' is not defined

In [341]:
pd_check.loc[5, 'channel_names'] = False

In [318]:
try:
    assert pd_check.values.all(), 'Error in continuity of recs'
except:
    logger.warn('Errors in continuity of recordings')
    print(pd_ceck[pd_check.where(False)])
    logger.warn('')
    raise

/home/zinch/conda/envs/msort/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  after removing the cwd from sys.path.
2018-12-19 12:43:01,427 - root - WARNING - Errors in continuity


AssertionError: Error in continuity of recs

In [313]:
pd_check.head(7)
pd_check.values.all()

False

In [336]:
pd_check[~pd_check.all(1)]

,channel_names,channels_sample_rate
5,False,True


In [328]:

pd_check[pd_check[['channel_names', 'channels_sample_rate']]==False]

,channel_names,channels_sample_rate
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,0.0,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [321]:
pd_check[pd_check==False]

,channel_names,channels_sample_rate
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,0.0,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [257]:
a = np.empty(0)
a == np.empty(0)

array([], dtype=bool)

In [253]:
apd.loc[0, 'channel_names']

array([b'D-004', b'D-006', b'D-008', b'D-012', b'D-019', b'D-022',
       b'D-023', b'D-024', b'D-025', b'D-026', b'D-027', b'D-028',
       b'D-029', b'ADC-00'], dtype='|S6')

In [265]:
pd_slice = apd['name'].isin(np.array([0, 5, 7]))
apd.loc[pd_slice]

,bit_depth,sample_rate,name,start_sample,start_time,is_multiSampleRate_data,channels_sample_rate,channel_bit_volts,channel_names,valid_samples,channel_names_diff
0,16,20000.0,0,0,1,0,"[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[14556240.0, 14556240.0, 14556240.0, 14556240....",False
5,16,20000.0,5,0,1,0,"[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[207360.0, 207360.0, 207360.0, 207360.0, 20736...","[True, True, True, True, True, True, True, Tru..."
7,16,20000.0,7,0,1,0,"[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[774720.0, 774720.0, 774720.0, 774720.0, 77472...","[True, True, True, True, True, True, True, Tru..."


In [274]:
cmplist = diff_arrays(apd, 'channel_names')
cmp_arr = np.stack(cmplist[1:])
cmp_arr.all(axis=1)

/home/zinch/conda/envs/msort/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [278]:
apd.head()

,bit_depth,sample_rate,name,start_sample,start_time,is_multiSampleRate_data,channels_sample_rate,channel_bit_volts,channel_names,valid_samples,channel_names_diff
0,16,20000.0,0,0,1,0,"[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[14556240.0, 14556240.0, 14556240.0, 14556240....",False
1,16,20000.0,1,0,1,0,"[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[106560.0, 106560.0, 106560.0, 106560.0, 10656...","[True, True, True, True, True, True, True, Tru..."
2,16,20000.0,2,0,1,0,"[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[1199520.0, 1199520.0, 1199520.0, 1199520.0, 1...","[True, True, True, True, True, True, True, Tru..."
3,16,20000.0,3,0,1,0,"[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[308160.0, 308160.0, 308160.0, 308160.0, 30816...","[True, True, True, True, True, True, True, Tru..."
4,16,20000.0,4,0,1,0,"[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[1199520.0, 1199520.0, 1199520.0, 1199520.0, 1...","[True, True, True, True, True, True, True, Tru..."


In [277]:
cmp_arr.shape

(186, 14)

In [135]:
rec_attr_dict['app_data']['channel_names'][:10]

array([b'D-004', b'D-006', b'D-008', b'D-012', b'D-019', b'D-022',
       b'D-023', b'D-024', b'D-025', b'D-026'], dtype='|S6')

In [ ]:
def 

## Scripts for sorting

### prep the files with their nice formats, locations and names


In [142]:
rhd_list

NameError: name 'rhd_list' is not defined

In [143]:
ss_folder = os.path.join(exp_base, sess_par['bird'], 'ss_data', sess_par['sess'])

file_paths, out_folder = msc.make_paths(ss_folder)
os.makedirs(ss_folder, exist_ok=True)
#todo:
# meta and pars from intan files et al
s_f = 20000

# parameters to pass to the msort scripts
ds_params = {'samplerate': s_f,
            'detect_sign': -1}

with open(file_paths['params'], 'w') as fp:
    json.dump(ds_params, fp)
    logger.info('Created session par files {}'.format(file_paths['params']))
    
# create the mda file from the rhd_list
#intan_meta, samples_in = intan.intan_to_bin(rhd_list[:], file_paths['mda'], variant='mda');


2018-12-18 18:33:33,063 - root - INFO - Created session par files /home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/params.json


In [144]:
ds_params

{'samplerate': 20000, 'detect_sign': -1}

In [145]:
file_paths

{'mda': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/raw.mda',
 'params': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/params.json',
 'geom': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/geom.csv',
 'filt': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/filt.mda.prv',
 'pre': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/pre.mda.prv',
 'firings': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/firings.mda',
 'firings_curated': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/firings_curated.mda',
 'cluster_metrics': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/cluster_metrics.json'}

In [146]:
# test mountainsort method for reading sorting parameters
msc.read_dataset_params(ss_folder)

{'samplerate': 20000, 'detect_sign': -1}

In [147]:
# Initialize a pipeline
from mountainlab_pytools import mlproc as mlp
Pipeline=mlp.initPipeline();

JSProxyWidget(status='Not yet rendered')

In [148]:
msc.sort_dataset?

Signature: msc.sort_dataset(*, file_paths:dict, freq_min:int=300, freq_max:int=6000, adjacency_radius:int=1, detect_threshold:float=3, dispatch_method:str='run', opts:dict={})
Docstring: <no docstring>
File:      ~/repos/pipefinch/pipefinch/neural/sort/mountain/core.py
Type:      function


In [149]:
output_dir = os.path.join(ss_folder, 'sort_out');
# Pipeline was failing, need to debug this ('NameError: name 'widgets' is not defined')
# with Pipeline:
#     msc.sort_dataset(file_paths=file_paths, adjacency_radius=-1, detect_threshold=3, dispatch_method='add')

msc.sort_dataset(file_paths=file_paths, adjacency_radius=-1, detect_threshold=2, freq_min=600);

2018-12-18 18:33:33,152 - pipefinch.sort.mountain.comre - INFO - Bandpass filter


RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/raw.mda --parameters freq_max:6000 freq_min:600 samplerate:20000 --outputs timeseries_out:/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/filt.mda.prv


KeyboardInterrupt: 

In [ ]:
file_paths

## Command for viewing:
 - open up terminal with the environment msort
 - go go the ss_data folder for the session
 - run the command: qt-mountainview --raw raw.mda --filt sort_out/filt.mda.prv --pre sort_out/pre.mda.prv --samplerate=20000 --firings sort_out/firings.mda --cluster_metrics sort_out/cluster_metrics.json